some helper functions

In [155]:
import pandas as pd
import numpy as np

def safe_div_s(a, b, b_min = 0.01):
    if (b > 0 or b < 0):
        return a / b
    else:
        return a / b_min
def safe_div(df, dividend_name, divisor_name, b_min = 0.01): #a safe division of two columns
    try:
        df[divisor_name]
    except:
        print ('Warning: A column name with the name must be specified =' + divisor_name)
        return None
    try:
        df[dividend_name]
    except:
        print ('Warning: A column name with the name must be specified =' + dividend_name)
        return None
    df.loc[df[divisor_name] == 0, divisor_name] = b_min
    df[dividend_name[:7]+'/'+divisor_name[:7]] = df[dividend_name]/df[divisor_name]
    return df
def median_calculated(df, filter_column, target_column): #it calculates a list of median values filterd by filter_column
    try:
        df[filter_column]
    except:
        print ('Warning: A column name with the name must be specified =' + filter_column)
        return None
    try:
        df[target_column]
    except:
        print ('Warning: A column name with the name must be specified =' + target_column)
        return None
    my_list = df[filter_column].unique()
    #print(my_list)
    #print(df.loc[df[filter_column] == my_list[0]])
    result = []
    for el in my_list:
        #print(df.loc[df[filter_column] == el])
        #print(df.loc[df[filter_column] == el][target_column])
        result.append(df.loc[df[filter_column] == el][target_column].median())
    return result
# it calculates dividend/divisor if divisor =0, it repaces it on median calculated with all rows with filter_columns

def median_safe_div(df, dividend_name, divisor_name, filter_column):
    try:
        df[divisor_name]
    except:
        print ('Warning: A column name with the name must be specified =' + divisor_name)
        return None
    try:
        df[dividend_name]
    except:
        print ('Warning: A column name with the name must be specified =' + dividend_name)
        return None
    try:
        df[filter_column]
    except:
        print ('Warning: A column name with the name must be specified =' + filter_column)
        return None
    my_list = df[filter_column].unique()
    #calculate basic
    new_name = dividend_name[:7]+'/'+divisor_name[:7]
    df[new_name] = df[dividend_name]/df[divisor_name]
    median =df[new_name].median()
    median_list = median_calculated(df, filter_column, new_name)
    j=0
    for el in median_list:
        if (el == np.inf or el == -np.inf):
            el = df[new_name].median()
            median_list[j] = df[new_name].median()
        j+=1    
    #print(median_list) 
    #try to replace inf
    #print(my_list)
    df[new_name].replace(to_replace=np.inf, value=median, inplace=True)
    print (df)
    print(df.loc[(df[filter_column] == 4)&((df[new_name]== np.inf) | (df[new_name]== np.inf))])
    #df.loc[(df[filter_column] == 4)&((df[new_name]== np.inf) | (df[new_name]== np.inf)), new_name] = 11111
    i=0
    for el in my_list:
        #df.loc[(df[filter_column] == 4)&((df[new_name]== np.inf) | (df[new_name]== np.inf)), new_name] = median_list[i]
        #print("i=", i, 'el=', el, 'median_list[i]=', median_list[i])
        #print(df)
        i+=1
        
    
    #df.loc[df[divisor_name] == 0, divisor_name] = aaaa(my_list, median_list, df[filter_column])
    #df[dividend_name[:7]+'/'+divisor_name[:7]] = df[dividend_name]/df[divisor_name]
    #print (df[new_name].median())
    return (df)
    
        
        

main_file_path = 'train.csv'
data = pd.read_csv(main_file_path)
pd.set_option('display.max_rows', 1000)
a = [[5, 1.2, 4.2], [1, 0, 0.03], [8, 5, 1], [8,2,4], [16,4,4], [32,9,4], [32,0,4]]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
#df['que'] = df['one'][(df['one'] >0) | (df['one'] <0)]
print(df)
print(median_safe_div(df, 'one', 'two', 'three'))
#print(safe_div2(df, 'one', 'two'))
#print(median_calculated(data, 'Neighborhood', 'LotArea'))
#print(median_calculated(df, 'one', 'two'))

print(data.describe())
print(data.columns)
unique_data = data['Neighborhood'].unique()
print(unique_data)
neighborhood = pd.get_dummies(data, columns=['Neighborhood'])
price_lot_data = safe_div(data['SalePrice'], data['LotArea'], 1)
print(price_lot_data)

   one  two  three
0    5  1.2   4.20
1    1  0.0   0.03
2    8  5.0   1.00
3    8  2.0   4.00
4   16  4.0   4.00
5   32  9.0   4.00
6   32  0.0   4.00
   one  two  three   one/two
0    5  1.2   4.20  4.166667
1    1  0.0   0.03  4.000000
2    8  5.0   1.00  1.600000
3    8  2.0   4.00  4.000000
4   16  4.0   4.00  4.000000
5   32  9.0   4.00  3.555556
6   32  0.0   4.00  4.000000
Empty DataFrame
Columns: [one, two, three, one/two]
Index: []
   one  two  three   one/two
0    5  1.2   4.20  4.166667
1    1  0.0   0.03  4.000000
2    8  5.0   1.00  1.600000
3    8  2.0   4.00  4.000000
4   16  4.0   4.00  4.000000
5   32  9.0   4.00  3.555556
6   32  0.0   4.00  4.000000
                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000

c:\users\mikle\lc101\datas\tf\lib\site-packages\pandas\core\series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]
c:\users\mikle\lc101\datas\tf\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U21') dtype('<U21') dtype('<U21')